In [3]:
import http.client, urllib, requests, urllib.request, urllib.parse, urllib.error, base64
import cognitive_face as CF 
from cognitive_face import util
import pandas as pd

In [4]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

/Users/Amiros/anaconda/lib/python3.5/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed autotime.py. To use it, type:
  %load_ext autotime


In [3]:
#https://github.com/Microsoft/Cognitive-Face-Python/blob/master/cognitive_face/face_list.py
#https://www.microsoft.com/cognitive-services/en-us/face-api/documentation/face-api-how-to-topics/howtoidentifyfacesinimage#step2-2

time: 728 µs


In [5]:
KEY = 'e3b77a2e317244f2a7198e9387190a64'  
CF.Key.set(KEY)

time: 1.45 ms


## Person Groups

In [6]:
#Create a person group
def create_group(person_group_id, name=None, user_data=None):
    name = person_group_id if name is None else name
    url = 'persongroups/{}'.format(person_group_id)
    json = {
        'name': name,
        'userData': user_data,
    }

    return util.request('PUT', url, json=json)

#Delete person group
def delete_person_group(person_group_id):
    url = 'persongroups/{}'.format(person_group_id)
    return util.request('DELETE', url)


# Lists person groups and their information
def lists(start=None, top=None):
    url = 'persongroups'
    params = {
        'start': start,
        'top': top,
    }

    return util.request('GET', url, params=params)


#Train a person group
def train(person_group_id):
    url = 'persongroups/{}/train'.format(person_group_id)
    
    return util.request('POST', url)


#Retrieve the training status of a person group
def get_status(person_group_id):
    url = 'persongroups/{}/training'.format(person_group_id)

    return util.request('GET', url)

time: 32.6 ms


In [6]:
# creating a person group for both candidates
Person_group_id = "candidates"

create_group(Person_group_id, name = 'candidates')

{}

time: 998 ms


## Persons

In [7]:
#Create a new person in a group
def create_person(person_group_id, name, user_data=None):
    
    url = 'persongroups/{}/persons'.format(person_group_id)
    json = {
        'name': name,
        'userData': user_data,
    }

    return util.request('POST', url, json=json)


#List all persons in a group
def list_persons(person_group_id):

    url = 'persongroups/{}/persons'.format(person_group_id)

    return util.request('GET', url)


#Retrieve a person's information, including registered persisted faces, `name` and `user_data`.
def get(person_group_id, person_id):

    url = 'persongroups/{}/persons/{}'.format(person_group_id, person_id)

    return util.request('GET', url)




# Add a representative face to a person for identification.
def add_face(image, person_group_id, person_id, user_data=None,
             target_face=None):
    
    url = 'persongroups/{}/persons/{}/persistedFaces'.format(
        person_group_id, person_id)
    headers, data, json = util.parse_image(image)
    params = {
        'userData': user_data,
        'targetFace': target_face,
    }

    return util.request('POST', url, headers=headers, params=params, json=json,
                        data=data)


#Retrieve information about a persisted face 
def get_face(person_group_id, person_id, persisted_face_id):

    url = 'persongroups/{}/persons/{}/persistedFaces/{}'.format(
        person_group_id, person_id, persisted_face_id)

    return util.request('GET', url)


#delete person
def delete_person(person_group_id, person_id):

    url = 'persongroups/{}/persons/{}'.format(person_group_id, person_id)

    return util.request('DELETE', url)

time: 69.7 ms


In [12]:
# Create persons for both candidates
#H_person = create_person(Person_group_id,"hillary")
#T_person= create_person(Person_group_id, "trump")

# get person IDs
H_personID = H_person['personId']
T_personID = T_person['personId']

NameError: name 'H_person' is not defined

time: 9.93 ms


In [11]:
list_persons('candidates')

[{'name': 'hillary',
  'persistedFaceIds': ['2fce2066-decc-4acd-9e42-7d7a8ed8630a',
   '60396af4-1110-4be4-b27d-e88da0a3781a',
   '731dc4ab-6ad9-48e1-85ba-4d5cd2c81ad8',
   '7c9f5fa8-7497-4de5-85c8-b1aa6a4d47a6',
   'a7d20073-4033-4232-a17a-cd169d65bb51',
   'c0c620ad-7c9d-4d47-a495-1c8002c5cc69',
   'ccb7b9dd-0dac-490c-a29c-3a1f9e582dae',
   'd196fde4-f013-4349-8aaf-14513cb44b6b',
   'f8a043a2-658a-4e7c-97dd-77e032d900d6',
   'fbcaa98a-0094-47a9-84b6-26df1b1e4011'],
  'personId': 'ca02317b-b2cf-4544-bcc4-6683d6634f59',
  'userData': None},
 {'name': 'trump',
  'persistedFaceIds': ['088d03ca-90fd-4448-ab73-15cfb839e042',
   '59d663ff-d48a-4ab8-bd15-a15518fa5beb',
   '86118b0d-864b-4450-9b2c-e5698f035af7',
   'b9ffa542-7967-4d4e-8e89-fbb62f45c251',
   'cd3554c0-6554-4094-aff5-1621ac5c8f8b',
   'dae45fa3-f09a-419d-9dc4-11a32da3e295',
   'e253a94a-6f91-40d0-bdf2-7256333325b8',
   'fd3e8ba1-d441-48e2-a971-ff7d38346f1c'],
  'personId': 'fa7e98e2-fac5-4d9d-affd-ebf4bb12bbd2',
  'userData': N

time: 673 ms


In [24]:
#Adding face to persons
image_t = []
for i in range (1,9):
    image_t = "https://s3-us-west-2.amazonaws.com/debateinemotion/sample/t/t%d.png" % i
    add_face(image_t, 'candidates', T_personID)
    
image_h = []
for i in range (1,11):
    image_h = "https://s3-us-west-2.amazonaws.com/debateinemotion/sample/h/h%d.png" % i
    add_face(image_h, 'candidates', H_personID)

time: 13.8 s


In [13]:
## Train
train('candidates')

{}

time: 1.28 s


## Verify candidaites face

In [14]:
def detect(image, face_id=True, landmarks=False, faceRectangle= False, attributes='gender'):
    
    url = 'detect'
    headers, data, json = util.parse_image(image)
    params = {
        'returnFaceId': face_id and 'true' or 'false',
        'returnFaceLandmarks': landmarks and 'true' or 'false',
        'returnFaceAttributes': attributes,
    }

    return util.request('POST', url, headers=headers, params=params, json=json,
                        data=data)

time: 5.64 ms


In [15]:
detect('https://s3-us-west-2.amazonaws.com/debateinemotion/sample/t/t1.png')

[{'faceAttributes': {'gender': 'male'},
  'faceId': 'ac856689-3314-4baf-93ba-7fa443c576a8',
  'faceRectangle': {'height': 142, 'left': 103, 'top': 93, 'width': 142}}]

time: 352 ms


In [ ]:
urlImage= ''

# Emtional Anlaysis function
def similar( json, data, headers, params ):

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result


headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json' 

json = { 'url': urlImage } 
data = None
params = None